In [1]:
import json
 
# Opening JSON file
f = open('./cubadebate.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)
f.close()
social_net = 'cubadebate'
document_dict = {}

In [7]:
from redis import Redis

vectorial_db = Redis(host='localhost', port=6379, db=6)


In [3]:
class Document:
    def __init__(self, text, url):
        self.text = text
        self.url = url
        self.total_term = 0
        self.f_max = 0
        self.vector = None

for obj in data:
    header = obj['text']
    url = obj['url']
    for c in obj['comments']:
        all_text = header + ' -> ' + c['text']
        hsh = hash(all_text)
        document_dict[hsh] = Document(all_text, url)

N = len(document_dict)
print(f"{N} document total")

8530 document total


In [2]:
import spacy

nlp = spacy.load("es_core_news_sm")
term_dict = {}

In [4]:
class Term:
    def __init__(self):
        self.doc_count = {}
    
    def add_match(self, hsh_doc):
        try: self.doc_count[hsh_doc] += 1
        except KeyError: self.doc_count[hsh_doc] = 1
    
    def match_by_document(self, hsh_doc):
        try: return self.doc_count[hsh_doc]
        except: return 0
    
    @property
    def match_len(self):
        return len(self.doc_count)

In [19]:
import progressbar
bar = progressbar.ProgressBar(maxval=N, \
    widgets=[progressbar.Bar('#', '[', ']'), ' ', progressbar.Percentage()])


In [ ]:

bar.start()
for i, key in enumerate(document_dict):
    doc = nlp(document_dict[key].text)
    count = 0
    for token in doc:
        if token.is_stop or token.is_punct or token.is_digit: continue

        count += 1
        try: term = term_dict[token.lemma_.lower()]
        except KeyError: term = term_dict[token.lemma_.lower()] = Term()

        term.add_match(key)
    
    for entity in doc.ents:

        try: term = term_dict[entity.text.lower()]
        except KeyError: term = term_dict[entity.text.lower()] = Term()
        count += 1
        term.add_match(key)
    
    bar.update(i+1)
    document_dict[key].total_term = count



bar.finish()


In [7]:
tn = len(term_dict)
print(f"{tn} term total")

47820 term total


In [8]:

bar.start()
for i, key in enumerate(document_dict):
    f_list = []
    for term in term_dict:
        f_list.append(term_dict[term].match_by_document(key))

    document_dict[key].f_max = max(f_list)
    bar.update(i+1)

bar.finish()


In [9]:
from math import log

def get_weight(word, hsh_doc):
    term = term_dict[word]
    doc = document_dict[hsh_doc]
    tf = term.match_by_document(hsh_doc)/ doc.f_max
    idf = log(N/term.match_len)

    return tf * idf

In [10]:
bar.start()
for i, key in enumerate(document_dict):
    vector = []
    for term in term_dict:
        vector.append(get_weight(term, key))

    document_dict[key].vector = vector
    bar.update(i+1)

bar.finish()

In [13]:
# bar.start()
for i, term in enumerate(term_dict):
    temp = {}
    for key in document_dict:
        try:w = document_dict[key].vector[i]
        except: w= 0
        if w != 0: temp[key] = w

    # bar.update(i+1)
    vectorial_db.set(term, str(temp))
# bar.finish()

bar.start()
for i, key in enumerate(document_dict):
    doc = document_dict[key]
    wd2 = sum([w*w for w in doc.vector])
    vectorial_db.set(key, str({
        'text': document_dict[key].text,
        'f_max': document_dict[key].f_max,
        'w2': wd2
    }))
    bar.update(i+1)
bar.finish()

    
vectorial_db.set('_$N$_', N)

True

In [63]:
query = "covid-19"

key = 'query'
query_term_dict = {}
doc = nlp(query)
count = 0

for token in doc:
    if token.is_stop or token.is_punct or token.is_digit: continue
    count += 1
    try: term = query_term_dict[token.lemma_.lower()]
    except KeyError: term = query_term_dict[token.lemma_.lower()] = Term()
    term.add_match(key)

for entity in doc.ents:
    try: term = query_term_dict[entity.text.lower()]
    except KeyError: term = query_term_dict[entity.text.lower()] = Term()
    count += 1
    term.add_match(key)



In [64]:
import ast

f_list = []
for term in query_term_dict:
    if vectorial_db.get(term) is None: continue
    f_list.append(query_term_dict[term].match_by_document(key))


fiq_max = max(f_list)
N = int(vectorial_db.get('_$N$_'))
q_vector = []
for word in query_term_dict:
    dict_ = vectorial_db.get(word)
    if dict_ is None: continue


    try: fi = query_term_dict[word].match_by_document(key)
    except KeyError:  fi = 0

    idf = log(N/len(ast.literal_eval(dict_.decode())))
    q_vector.append((0.5 + 0.5 * fi/fiq_max) * idf)


In [65]:
a = vectorial_db.keys()

document_list = []
for key in a:
    try: document_list.append(int(key))
    except ValueError: 
        pass


In [66]:
import progressbar
bar = progressbar.ProgressBar(maxval=len(document_list), \
    widgets=[progressbar.Bar('#', '[', ']'), ' ', progressbar.Percentage()])

wq2 = sum([w*w for w in q_vector])

t_dict = {}
for term in query_term_dict:
    dict_ = vectorial_db.get(term)
    if dict_ is None : t_dict[term] = dict_
    else: t_dict[term] = ast.literal_eval(dict_.decode())

bar.start()
ranking_list = []
for i, tuple_ in enumerate(zip(document_list, vectorial_db.mget(document_list))):
    doc_vector = []
    key, doc = tuple_
    if doc is None: continue
    doc = ast.literal_eval(doc.decode())
    if not 'w2' in doc: continue

    for term in query_term_dict:
        dict_ = t_dict[term]
        if dict_ is None: continue

        try: doc_vector.append( dict_[key]/doc['f_max'] )
        except: doc_vector.append(0)

    wij = sum([d*q for d, q in zip(doc_vector, q_vector)])

    ranking_list.append((wij/(wq2* doc['w2']), doc['text']))
    bar.update(i+1)

bar.finish()


ranking_list.sort(key=lambda x: x[0], reverse=True)
i = 0

[########################################################################] 100%


In [76]:
_, doc = ranking_list[i]
text, comment = doc.split('->') 

for term in query_term_dict:
    text = text.replace(term, "\33[92m{}\033[00m".format(term))
    comment = comment.replace(term, "\33[92m{}\033[00m".format(term))
print(text)
print("\n#####################################################################\n")
print(comment)
i += 1

En correspondencia con la situación epidemiológica internacional y nacional de la COVID-19 y los niveles de inmunización alcanzados, se ha decidido realizar las siguientes modificaciones a los requisitos de entrada al país como parte del Control Sanitario Internacional. De acuerdo con Francisco Durán García, director nacional de Epidemiología del Minsap, las medidas de flexibilización entrarán en vigor el 6 de abril.  

#####################################################################

 Estoy en las mismas...


In [43]:
_, doc = ranking_list[i]
text, comment = doc.split('->') 

for term in query_term_dict:
    text = text.replace(term, "\33[92m{}\033[00m".format(term))
    comment = comment.replace(term, "\33[92m{}\033[00m".format(term))
print(text)
print("\n#####################################################################\n")
print(comment)
i += 1

Teniendo en cuenta la situación existente con la disponibilidad de productos y con el objetivo de hacer más viable la venta, lograr mayor equidad y por consiguiente disminuir la aglomeración de personas en los establecimientos, se ha decidido adoptar un grupo de medidas, entre las que se encuentran: Estas medidas se ponen en vigor a partir del próximo jueves 21 de abril y en correspondencia con las características de cada municipio, los grupos municipales organizarán su sistema de trabajo y de control, para ir aplicando cada medida y estableciendo el procedimiento para lograr el objetivo previsto.  

#####################################################################

 Una nueva medida que afecta al trabajador pues no hay tiempo para comprar en lo que te trasladada al lugar de residencia. Hasta cuándo nuestros dirigentes de La Habana van a seguir tomando medidas que afecten al pueblo revolucionario,es hora de poner los pies en la tierra y pensar en las consecuencias de lo que hacen o

In [44]:
_, doc = ranking_list[i]
text, comment = doc.split('->') 

for term in query_term_dict:
    text = text.replace(term, "\33[92m{}\033[00m".format(term))
    comment = comment.replace(term, "\33[92m{}\033[00m".format(term))
print(text)
print("\n#####################################################################\n")
print(comment)
i += 1

Teniendo en cuenta la situación existente con la disponibilidad de productos y con el objetivo de hacer más viable la venta, lograr mayor equidad y por consiguiente disminuir la aglomeración de personas en los establecimientos, se ha decidido adoptar un grupo de medidas, entre las que se encuentran: Estas medidas se ponen en vigor a partir del próximo jueves 21 de abril y en correspondencia con las características de cada municipio, los grupos municipales organizarán su sistema de trabajo y de control, para ir aplicando cada medida y estableciendo el procedimiento para lograr el objetivo previsto.  

#####################################################################

 Yo estoy de acuerdo en que pongan los productos por bodegas y carnicerías, porque los que trabajamos no podemos adquirir nada. Además en las tiendas siempre son los mismos en las colas, no entiendo porque si se debe apuntar en la libreta de abastecimiento y toca cada cierto tiempo. El problema es que no les apuntan y v

In [45]:
_, doc = ranking_list[i]
text, comment = doc.split('->') 

for term in query_term_dict:
    text = text.replace(term, "\33[92m{}\033[00m".format(term))
    comment = comment.replace(term, "\33[92m{}\033[00m".format(term))
print(text)
print("\n#####################################################################\n")
print(comment)
i += 1

Teniendo en cuenta la situación existente con la disponibilidad de productos y con el objetivo de hacer más viable la venta, lograr mayor equidad y por consiguiente disminuir la aglomeración de personas en los establecimientos, se ha decidido adoptar un grupo de medidas, entre las que se encuentran: Estas medidas se ponen en vigor a partir del próximo jueves 21 de abril y en correspondencia con las características de cada municipio, los grupos municipales organizarán su sistema de trabajo y de control, para ir aplicando cada medida y estableciendo el procedimiento para lograr el objetivo previsto.  

#####################################################################

 Pienso que sería bueno la municipalizacion e incluso debería realizarse por bodega pq la masa trabajadora que no tenemos tiempo para colas tenemos que comprar más que sobreprecio y mantener a los coleros o no comer. Si toda la epidemia no hemos podido comprar pq no establecer que todos los núcleos podamos comprar aunqu

In [46]:
_, doc = ranking_list[i]
text, comment = doc.split('->') 

for term in query_term_dict:
    text = text.replace(term, "\33[92m{}\033[00m".format(term))
    comment = comment.replace(term, "\33[92m{}\033[00m".format(term))
print(text)
print("\n#####################################################################\n")
print(comment)
i += 1

Teniendo en cuenta la situación existente con la disponibilidad de productos y con el objetivo de hacer más viable la venta, lograr mayor equidad y por consiguiente disminuir la aglomeración de personas en los establecimientos, se ha decidido adoptar un grupo de medidas, entre las que se encuentran: Estas medidas se ponen en vigor a partir del próximo jueves 21 de abril y en correspondencia con las características de cada municipio, los grupos municipales organizarán su sistema de trabajo y de control, para ir aplicando cada medida y estableciendo el procedimiento para lograr el objetivo previsto.  

#####################################################################

 Bienvenida la organización, hace mucho tiempo debieron hacerlo. Pero a nuestro entender por que no en las bodegas???? Para garantizar que los negocios q existen en estos momentos, producto de la necesidad, se minimicen y podamos alcanzar los productos con precios asequibles


In [47]:
_, doc = ranking_list[i]
text, comment = doc.split('->') 

for term in query_term_dict:
    text = text.replace(term, "\33[92m{}\033[00m".format(term))
    comment = comment.replace(term, "\33[92m{}\033[00m".format(term))
print(text)
print("\n#####################################################################\n")
print(comment)
i += 1

En correspondencia con la situación epidemiológica internacional y nacional de la COVID-19 y los niveles de inmunización alcanzados, se ha decidido realizar las siguientes modificaciones a los requisitos de entrada al país como parte del Control Sanitario Internacional. De acuerdo con Francisco Durán García, director nacional de Epidemiología del Minsap, las medidas de flexibilización entrarán en vigor el 6 de abril.  

#####################################################################

 Mi pregunta es cuándo se podrá dejar de usar la mascarilla todo el tiempo, considero que para entrar a tiendas o establecimientos públicos es necesaria Perooooooo que no sea de carácter obligatorio todo el tiempo. Ya en muchos países la mascarilla es opcional.
